# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [2]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [3]:
df = pd.read_excel(r'C:\Users\user\предпосылки\Экс4.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,9.000000,7.000,7.000000,6.0,5.0,6
1,Экспертиза промышленной безопасности,0.111111,1.000000,9.000,8.000000,9.0,1.0,7
2,Готовность к ликвидации последствий аварий и и...,0.142857,0.111111,1.000,1.000000,8.0,6.0,7
3,Оформление информационными знаками и знаками б...,0.142857,0.125000,1.000,1.000000,9.0,7.0,1
4,Содержание объектов и оборудования,0.166667,0.111111,0.125,0.111111,1.0,7.0,9


In [4]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [5]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [6]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              9.000000   
1       0.111111                              1.000000   
2       0.142857                              0.111111   
3       0.142857                              0.125000   
4       0.166667                              0.111111   
5       0.200000                              1.000000   
6       0.166667                              0.142857   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           7.000000         
1                                           9.000000         
2                                           1.000000         
3                                           1.000000         
4                                           0.125000         
5                                           0.166667         
6                                           0.142857         

   Оформление информационными знаками 

In [7]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [8]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [9]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.000000,9.000000,7.000000,7.000000,6.000000,5.000000
1,0.111111,1.000000,9.000000,8.000000,9.000000,1.000000
2,0.142857,0.111111,1.000000,1.000000,8.000000,6.000000
3,0.142857,0.125000,1.000000,1.000000,9.000000,7.000000
4,0.166667,0.111111,0.125000,0.111111,1.000000,7.000000
5,0.200000,1.000000,0.166667,0.142857,0.142857,1.000000
6,0.166667,0.142857,0.142857,1.000000,0.111111,0.111111


In [10]:
DF_1=df[:].copy()

In [11]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  1.930159
Экспертиза промышленной безопасности                         11.490079
Готовность к ликвидации последствий аварий и инцидентов      18.434524
Оформление информационными знаками и знаками безопасности    18.253968
Содержание объектов и оборудования                           33.253968
Охранные зоны (ОЗ)                                           27.111111
Организация рабочего процесса и безопасности персонала       40.000000
dtype: float64

In [12]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [13]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,9.000000,7.000,7.000000,6.0,5.0,6,0.518092,0.783284,0.379722,0.383478,0.180430,0.184426,0.150
1,0.111111,1.000000,9.000,8.000000,9.0,1.0,7,0.057566,0.087032,0.488214,0.438261,0.270644,0.036885,0.175
2,0.142857,0.111111,1.000,1.000000,8.0,6.0,7,0.074013,0.009670,0.054246,0.054783,0.240573,0.221311,0.175
3,0.142857,0.125000,1.000,1.000000,9.0,7.0,1,0.074013,0.010879,0.054246,0.054783,0.270644,0.258197,0.025
4,0.166667,0.111111,0.125,0.111111,1.0,7.0,9,0.086349,0.009670,0.006781,0.006087,0.030072,0.258197,0.225


In [14]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.518092,0.783284,0.379722,0.383478,0.180430,0.184426,0.150
1,0.057566,0.087032,0.488214,0.438261,0.270644,0.036885,0.175
2,0.074013,0.009670,0.054246,0.054783,0.240573,0.221311,0.175
3,0.074013,0.010879,0.054246,0.054783,0.270644,0.258197,0.025
4,0.086349,0.009670,0.006781,0.006087,0.030072,0.258197,0.225
5,0.103618,0.087032,0.009041,0.007826,0.004296,0.036885,0.225
6,0.086349,0.012433,0.007749,0.054783,0.003341,0.004098,0.025


In [15]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.518092,0.783284,0.379722,0.383478,0.180430,0.184426,0.150,0.368490
1,0.057566,0.087032,0.488214,0.438261,0.270644,0.036885,0.175,0.221943
2,0.074013,0.009670,0.054246,0.054783,0.240573,0.221311,0.175,0.118514
3,0.074013,0.010879,0.054246,0.054783,0.270644,0.258197,0.025,0.106823
4,0.086349,0.009670,0.006781,0.006087,0.030072,0.258197,0.225,0.088879


In [16]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0000000000000002

In [17]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [18]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

4.981043168282563
3.2454334081398093
1.6134534646836893
1.6070120689407354
0.9244480493155198
0.6601338275146323
0.26194846086987983


In [19]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

13.293472447746828


In [20]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

1.0489120746244713
1.4142857142857144
0.7416550022597271


In [21]:
# Получилось, что значение CR=0.74, что больше 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами